<table class="table table-bordered">
    <tr>
        <th style="text-align:center; width:35%"><img src='https://dl.dropbox.com/s/qtzukmzqavebjd2/icon_smu.jpg' style="width: 300px; height: 90px; "></th>
    <th style="text-align:center;"><font size="4"> <br/>IS.215 - Analytics in Python Practical 1</font></th>
    </tr>
</table> 

This program builds a classifier for Pima Indians Diabetes dataset - https://www.kaggle.com/uciml/pima-indians-diabetes-database. It is a binary (2-class) classification problem. There are 768 observations with 8 input variables and 1 output/target variable. The variable names are as follows:

- Number of times pregnant.
- Plasma glucose concentration a 2 hours in an oral glucose tolerance test.
- Diastolic blood pressure (mm Hg).
- Triceps skinfold thickness (mm).
- 2-Hour serum insulin (mu U/ml).
- Body mass index (weight in kg/(height in m)^2).
- Diabetes pedigree function.
- Age (years).
- Target variable (0 -'no' or 1-'yes').

**Step 1: import relevant libraries**

In [1]:
#Step 1: import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import MinMaxScaler

**Step 2: read in the data**

After importing the libraries, the source file 'diabetes.csv' is imported by making use of read_csv of pandas into dataframe (df). Essentially it is reading the data as dataframe columns with the corresponding name for ease of reference.

The preliminary analysis includes checking for dimension of the imported dataframe (df.shape) and making sure that it is 768 rows and 9 columns (including the target column). df.describe() is used to check the summary of the individual column.

In [2]:
#Step 2: read in the data and do a preliminary analysis

df = pd.read_csv("diabetes.csv", names=['num_pregnant','glucose_conc','diastolic_bp','triceps_thick','serum_insulin','bmi','pedigree','age','class'])
print(df.shape)
print(df.describe())

(768, 9)
       num_pregnant  glucose_conc  diastolic_bp  triceps_thick  serum_insulin  \
count    768.000000    768.000000    768.000000     768.000000     768.000000   
mean       3.845052    120.894531     69.105469      20.536458      79.799479   
std        3.369578     31.972618     19.355807      15.952218     115.244002   
min        0.000000      0.000000      0.000000       0.000000       0.000000   
25%        1.000000     99.000000     62.000000       0.000000       0.000000   
50%        3.000000    117.000000     72.000000      23.000000      30.500000   
75%        6.000000    140.250000     80.000000      32.000000     127.250000   
max       17.000000    199.000000    122.000000      99.000000     846.000000   

              bmi    pedigree         age       class  
count  768.000000  768.000000  768.000000  768.000000  
mean    31.992578    0.471876   33.240885    0.348958  
std      7.884160    0.331329   11.760232    0.476951  
min      0.000000    0.078000   21.00

**Step 3: create input and target data**

Since the data is read as a whole into df. There is a need to split the content that containing input variables or information regarding the diagnotics values and the target value. The target value is actually the label of the input values, indicating if the person has diabetes or not.

input_df = df.drop('class', axis=1) is to create input_df that contains only the first 8 columns containing diagnotics information and removing the last column ('class'). It doesn't make sense to include 'class' in the data to train the model since it is the target we want to learn and predict.

target = df['class'] is creating a dataframe that only contains the target value - '0' or '1' 

In [3]:
#Step 3: Split into input_df and target dataframe. axis=0, row, axis=1, column.
input_df = df.drop('class', axis=1)
target = df['class']

print(input_df.shape,target.shape)

(768, 8) (768,)


In [4]:
#distribution of class - imbalance class with '1' having lesser count
target.value_counts()

0    500
1    268
Name: class, dtype: int64

**Step 4: split data into training and testing data**

In order to train a classifier or a classification model, there is a need to split the input data and corresponding target value into two parts - training and testing data. The purpose of having two set of data is to evaluate the model (built using the training data). In other words, we want to be able to evaluate how good is the model by comparing the predicted result from the trained model and the testing data. Scikit-learn provides a useful function - train_test_split that is able to separate two set of data according to a proportion, for example, when test_size=0.3, it means splitting 70% training -30% testing. 

It is common to use 'X' to denote input values and 'y' to indicate target value in scikit-learn. As a result, X_train, y_train is essentially the input values and corresponding target value of training data. Similarly for X_test and y_test for testing data.

In [5]:
#Step 4: Split feature and label sets to train and data sets - 70-30, random_state is desirable for reproducibility, stratify - same proportion as input data

X_train, X_test, y_train, y_test = train_test_split(input_df, target, test_size = 0.3, random_state = 10, stratify = target)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(537, 8) (231, 8) (537,) (231,)


Since the range of input values are of different scale (from df.describe()), there is a need to normalize the values between 0-1 so that the range of values won't influence the model. It is important to do the same scaling and transforming on both input values of training data (X_train) and testing data (X_test) to ensure meaningful evaluation.

In [6]:
#Question 2 - Normalize using MinMaxScaler to constrain values to between 0 and 1.

scaler = MinMaxScaler(feature_range = (0,1))

scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**Step 5: create the Logistic Regression model**

The algorithm used in this example is Logistic Regression. It is one of the commonly used classification algorithm. The model created is 'logreg'. You can use any other variable to represent it, e.g., 'lr'. 

logreg.fit(X_train, y_train) - using the training input data and target value to build the 'logreg' model. After this command, 'logreg' will learn the 'rules'/'knowledge' of differentiating an onset diabetes and non-diabetes patient.

y_pred = logreg.predict(X_test) - 'logreg' model is used to predict X_test and the predicted result can be found in y_pred. If the model has learnt all the 'knowledge/rules', y_pred will be 100% match with y_test. 

The subsequent codes are to evaluate the predicted results using both accuracy and F1-score. You may observe that the recall value is quite low for target value '1', this is likely due to the imbalance class '1' and '0' of the original data with '1' having 268 and '0' having 500 records.

In [7]:
#Step 5: Create a logistic regression classifier, default c=1

logreg = LogisticRegression(solver='liblinear')
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))

#look at the value under the '1' class (or 'yes') for the corresponding precision, recall and f1 score
print(classification_report(y_test, y_pred))

Testing accuracy 0.8051948051948052
              precision    recall  f1-score   support

           0       0.80      0.93      0.86       150
           1       0.81      0.58      0.68        81

    accuracy                           0.81       231
   macro avg       0.81      0.75      0.77       231
weighted avg       0.81      0.81      0.80       231



One approach to improve the model is to handle the imbalance class data by oversampling the minority class. The method used in this example is SMOTE which stands for Synthetic Minority Over-sampling Techniques. SMOTE is applied on the X_train and y_train to ensure that both '1' and '0' classes are represented equally.

Based on the result, it can be observed that precision and recall are now more balanced (for the 'yes' or '1' class) and this model created is a better model to be used in predicting the onset of diabates. 

In [9]:
#Question 4 -Rerunning above with resampled data - using oversampling

sm = SMOTE(random_state = 7)
X_train_sm, y_train_sm = sm.fit_sample(X_train, y_train)

print(pd.value_counts(pd.Series(y_train_sm)))

clf = logreg.fit(X_train_sm, y_train_sm)
#score() is one way to get the model accuracy from scikit_learn logistic regression. Take note that the input parameter is X_test(data) and y_test(label)
print('Model accuracy is',clf.score(X_test, y_test))

y_pred = clf.predict(X_test)
#accuracy_score() can also be used to get the model accuracy - similar as above. Take note that the input parameter is y_test(label), y_pred(predicted label).
#print('Model accuracy is %s' % accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

1    350
0    350
Name: class, dtype: int64
Model accuracy is 0.7359307359307359
              precision    recall  f1-score   support

           0       0.84      0.73      0.78       150
           1       0.60      0.75      0.67        81

    accuracy                           0.74       231
   macro avg       0.72      0.74      0.72       231
weighted avg       0.76      0.74      0.74       231



It is important to be able to interpret the model created and to understand the importance of each features. For Logistic Regression, since we have normalized the input values, we can make use of the magnitude of the coefficients (coef_) to find the important features. 

argsort returns indices of the sorted values from smallest to the largest. By adding a '-' for the logreg.coef_, it will sort in descending order.

Feature with largest value contributes the most to the model (based on the magnitude associated with the feature, it is glucose_conc).

In [10]:
#Question 1,3 - get the descending sorted indices based on coefficient values
sorted_index = np.argsort(-logreg.coef_)

#get the feature_names
feature_names = input_df.columns

#get the names of the important features (largest to smallest)
print(feature_names.to_numpy()[sorted_index])

[['glucose_conc' 'bmi' 'pedigree' 'num_pregnant' 'age' 'serum_insulin'
  'triceps_thick' 'diastolic_bp']]
